In [95]:
url='https://www.tsx.com/files/trading/interlisted-companies.txt'
sectors = pd.read_csv(url, delimiter="\t", encoding='utf-8', header=1)
sectors.head(4)

,Symbol,Name,US Symbol,Sector,International Market
0,ABST:TSX,Absolute Software Corporation,ABST,Technology,NasdaqGM
1,ABX:TSX,Barrick Gold Corporation,GOLD,Mining,NYSE
2,ACB:TSX,Aurora Cannabis Inc.,ACB,Life Sciences,NasdaqGS
3,ACST:TSXV,Acasti Pharma Inc.,ACST,Life Sciences,NasdaqCM


In [103]:
from bs4 import BeautifulSoup as bs
import requests
import unicodedata
from string import ascii_uppercase
def get_info(link):
    r = requests.get(link)
    soup = bs(r.content)
    info_tables = soup.findAll('table', attrs={'class': 'infobox vcard'})
    
    if len(info_tables) > 0:
        rows = info_tables[0].findAll('tr')

        info = {}
        for row in rows:
            k = row.find('th')
            if k and k.text != None:
                k = unicodedata.normalize('NFKD', k.text).strip().replace('\n', ' ')

            v = row.find('td')
            if v and v.text != None:
                v = unicodedata.normalize('NFKD', v.text).strip().replace('\n', ' ')
            else:
                v = 'N/A'


            if k != None:
                # print(k + ': ', v)
                info[k] = v
        return info

def get_company_links(company_links_page):
    r = requests.get(company_links_page)
    soup = bs(r.content)
    a_list = soup.findAll('table', attrs={'class': None})[0].findAll('a', href = lambda h: h and 'wiki' in h)
    return ['https://en.wikipedia.org' + a['href'] for a in a_list]

company_links_pages = ['https://en.wikipedia.org/wiki/Companies_listed_on_the_Toronto_Stock_Exchange_({})'.format(alphabet) for alphabet in ascii_uppercase]

links = []

for page in company_links_pages:
    links = links + get_company_links(page)

infos = [get_info(link) for link in links if get_info(link) != None]
infos

import json
with open('company_description.json', 'w', encoding='utf-8') as f:
    json.dump(infos, f, ensure_ascii=False, indent=4)


In [5]:
from bs4 import BeautifulSoup
import requests
import unicodedata

In [21]:
def getTSX():
    ## Identify table by content; Scrape with BS4
    content = requests.get("https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index").content
    soup = BeautifulSoup(content,'html.parser')
    table = soup.find(text='Ticker').find_parent('table')

    ## Extract all elements from each row in table; Append to listRows
    listRows = []
    for row in table.find_all('tr')[1:]:
        listRows.append([cell.get_text(strip=True) for cell in row.find_all('td')])

    ## Convert list of lists to df; Standardize values to match SPY; Add .TO suffix to tickers
    df = pd.DataFrame(listRows)
    df.columns = ['Symbol','Company','Sector','Industry']
    df['Exchange'] = 'TSX'
    #df['Sector'] = df.apply(standardizeSector,axis=1)
    df['Symbol'] = df['Symbol'] + '.TO'
    
    return df


In [22]:
import pandas as pd
d = getTSX()
d

,Symbol,Company,Sector,Industry,Exchange
0,ARE.TO,Aecon Group Inc.,Industrials,Construction,TSX
1,AEM.TO,Agnico Eagle Mines Limited,Basic Materials,Metals & Mining,TSX
2,AC.TO,Air Canada,Industrials,Transportation,TSX
3,AGI.TO,Alamos Gold Inc.,Basic Materials,Metals & Mining,TSX
4,AQN.TO,Algonquin Power & Utilities Corp.,Utilities,Independent Power Producers,TSX
...,...,...,...,...,...
232,WCP.TO,Whitecap Resources Inc.,Energy,Oil & Gas,TSX
233,WPK.TO,Winpak Ltd.,Consumer Cyclical,Packaging & Containers,TSX
234,WIR.UN.TO,WPT Industrial Real Estate Investment Trust,Real Estate,REITs,TSX
235,WSP.TO,WSP Global Inc.,Industrials,Construction,TSX
